![Astrofisica Computacional](../../new_logo.png)

### Applied Exercise Set: Numerical Differentiation in Computational Astrophysics

# Mathematical Foundation: Finite Difference Refresher



Numerical differentiation serves as the essential bridge between theoretical physics—often expressed as continuous differential equations—and the discrete datasets obtained through observations or high-performance simulations. In computational astrophysics, we approximate derivatives by constructing "stencils" that sample the function at discrete grid points.

While these formulas can be derived via Lagrange Interpolation (approximating the function with a polynomial that passes through specific data points), they are most commonly conceptualized using Taylor Series Expansions. By discretizing the domain into a constant grid spacing h (the interval between samples), we can define the following first-order derivative approximations:

Method Name	Discrete Formula ($f'_i \approx$)	Truncation Error\
Forward Difference $(D_+)	\frac{f_{i+1} - f_i}{h}	O(h)$\
Backward Difference $(D_-)	\frac{f_i - f_{i-1}}{h}	O(h)$\
Centered Difference $(D_0)	\frac{f_{i+1} - f_{i-1}}{2h}	O(h^2)$

Note on Grid Spacing (h) and Symmetry: The parameter h represents the discretization step (e.g., $\Delta t$, $\Delta r$, or $\Delta D$). Note that the Centered Difference achieves a higher order of accuracy ($O(h^2)$) than one-sided methods. This occurs because the symmetry of the centered stencil causes the $h^1$ error terms in the Taylor expansion to cancel out exactly, leaving the $h^2$ term as the leading order of error.


--------------------------------------------------------------------------------



# Exercise 1: Pulsar Timing and Orbital Acceleration



Context: Pulsars are highly magnetized, rotating neutron stars. By analyzing the Doppler shift in their pulse arrival times, we can measure their line-of-sight velocity $v(t)$. In a binary system, the rate of change of this velocity provides the orbital acceleration, $a(t) = \frac{dv}{dt}$.

Dataset: The following table provides 5 velocity points for a pulsar with a constant time step $h = 0.5$ seconds.


| Index (i) | Time $t$ (s) | Velocity $v$ (km/s) |
|---|---|---|
| 0 | 0.0 | 200.0 |
| 1 | 0.5 | 205.2 |
| 2 | 1.0 | 210.8 |
| 3 | 1.5 | 216.9 |
| 4 | 2.0 | 223.5 |




Tasks:

* Task A: Calculate the acceleration at the initial point ($t_0 = 0.0$ s) using the Forward Difference formula.
* Task B: Calculate the acceleration at the final point ($t_4 = 2.0$ s) using the Backward Difference formula.
* Task C: Calculate the acceleration at the interior point ($t_2 = 1.0$ s) using the Centered Difference formula.


--------------------------------------------------------------------------------


In [5]:
time = np.array([0.0, 0.5, 1.0, 1.5, 2.0])
vel = np.array([200.0, 205.2, 210.8, 216.9, 223.5])
h = 0.5 

def F_der(index, h):
    return (vel[index + 1] - vel[index]) / h 


def B_der(index, h):     
    return (vel[index] - vel[index-1]) / h 


def C_der(index, h):
    return ( vel[index + 1] - vel[index - 1] ) / (2.0 * h)



print(f"Task A: {F_der(0, h)}")
print(f"Task B: {B_der(4, h)}")
print(f"Task C: {C_der(2, h)}")


Task A: 10.399999999999977
Task B: 13.199999999999989
Task C: 11.700000000000017




# Exercise 2: Stellar Structure and Density Gradients



Context: In stellar modeling, hydrostatic equilibrium is dictated by the pressure gradient $\frac{dP}{dr}$. When simulating the interior of a star, we often encounter a physical boundary at the surface. At the outermost data point of a model, we cannot use centered or forward differences because there is no physical data "outside" the star's boundary. This constraint necessitates the use of one-sided, multi-point stencils to maintain high accuracy.

Dataset: Radius $r$ (in $10^3$ km) and Pressure $P$ (arbitrary units) with a step $h = 0.1 \times 10^3$ km.


| Index (i) | Radius r | Pressure P |
|---|---|---|
| 0 | 1.0 | 10.00 |
| 1 | 1.1 | 9.15 |
| 2 | 1.2 | 8.20 |
| 3 | 1.3 | 7.12 |
| 4 | 1.4 | 5.90 |


Task: Calculate the pressure gradient $\frac{dP}{dr}$ at the outermost radius ($r = 1.4$) using the Three-Point Second-Order Backward Difference formula ($D_{-2}$): $f'(x_i) \approx \frac{3f_i - 4f_{i-1} + f_{i-2}}{2h}$

Instructional Guidance: Substitute the values for $P_4$, $P_3$, and $P_2$ into the stencil. In your conclusion, explain why this $O(h^2)$ method is preferred over a simple $O(h)$ backward difference. Hint: Consider the convergence rate—how the error would scale if you were to reduce the step size h by a factor of 10.


--------------------------------------------------------------------------------



In [6]:
data = np.array([ 5.90, 7.12, 8.20 ])[::-1]
h = 0.1

def der(index, h):
    return (3.0 * data[index] - 4.0 * data[index - 1] + data[index - 2]) / (2.0 * h)


print(f"Task: {der(2, 0.1)}")

Ex 2: -12.899999999999991


First of all, this method is more precise: the mere fact we're using more information to get a stimation of the derivate tells us that this method is potentially more precise. And about the errors, if we decrease $h$ in a factor of $10$, hte TPSOBD error will decrease in an order of 100, while the simple BD only in a factor of 10.


# Exercise 3: Galactic Dynamics and Redshift Gradients



Context: By mapping the redshift $z$ of galaxies at various distances D within a cluster, astronomers can determine the velocity field and the rate of expansion within the cluster. We require both the first derivative $\frac{dz}{dD}$ and the second derivative $\frac{d^2z}{dD^2}$ (the gradient of the gradient) to model the cluster's mass distribution.

Dataset: Distance step $h = 2$ Mpc.


| Index (i) | Distance D (Mpc) | Redshift z |
|---|---|---|
| 0 | 10 | 0.0020 |
| 1 | 12 | 0.0035 |
| 2 | 14 | 0.0055 |
| 3 | 16 | 0.0080 |
| 4 | 18 | 0.0110 |



Tasks:

* Task A: Apply the Centered Difference formula to approximate the first derivative $\frac{dz}{dD}$ at the midpoint ($D = 14$ Mpc).
* Task B: Using the standard $O(h^2)$ centered formula for second derivatives (Reference: Table 2 of FiniteDifferences.pdf), approximate $\frac{d^2z}{dD^2}$ at the midpoint ($D = 14$ Mpc): $f''(x_i) \approx \frac{f_{i+1} - 2f_i + f_{i-1}}{h^2}$

Requirement: Show the step-by-step substitution of the values $z_1$, $z_2$, $z_3$ into the formulas below.


* Substitution for $\frac{dz}{dD}$: 
* Substitution for $\frac{d^2z}{dD^2}$:
  
_This can be easily seen in the code, how these values are placed, so no need to rewrite them here_


--------------------------------------------------------------------------------


In [7]:

F = np.array([0.0035, 0.0055, 0.0080])
h = 2.0


def first_OD(index, dx):
    return (F[index + 1] - F[index - 1]) / (2.0*dx)

def second_OD(index, dx):
    return ( F[index+1] - 2.0*F[index] + F[index-1]) / (dx**2)ú


print(f"Task A: {first_OD(1, h)}")
print(f"Task B: {second_OD(1, h)}")



Task A: 0.0011250000000000001
Task B: 0.00012500000000000022




# Error Analysis and Consistency Discussion




As a numerical analyst, you must evaluate the fidelity of your discretization. Based on the provided source context, answer the following:

1. Theoretical Accuracy: Why is the Centered Difference method in Exercise 3 significantly more reliable than the Forward Difference used in Exercise 1? Reference the Taylor series truncation in your answer.
2. Convergence and Step Size: If the step size $h$ in the stellar model (Exercise 2) were halved, by what factor would the truncation error decrease for this second-order ($p=2$) method? Use the $O(h^p)$ relationship to justify your answer.
3. High-Fidelity Simulations: In simulations of galactic collisions or long-term planetary dynamics, "Mixed Differences" or "Approximations with more points" (such as the $O(h^4)$ coefficients $a_1=2/3$, $a_2=-1/12$ found in diferencias_compactas.pdf) are utilized. Explain why O(h) or O(h^2) methods are often insufficient for these specific astrophysical applications.


--------------------------------------------------------------------------------



**Solution**

1. Because we're using a truncation of order 2 instead of order 1, that means that we have a precision gain of 1 order when using central difference than Forward difference (that is, $p_{central} > p_{forward}$)
2. As we're using a $p=2$, when $h = 1$ then $O(h^p) \propto 1.0$ while, when halving it, $O((\frac{h}{2})^p) \propto \frac{1}{4}$
3. If we're dealing with functions with rapid fluctuations or functions with messy behaviours, using single-order numerical differences is not really useful, not necessarily because of the precision $O(h)$ or $O(h^2)$, but mostly because using fewer points (big partitions) may place the points we're evaluating the derivatives at in regions where we cannot properly infer any sort of oscilating behavior. 


# Formula Key



The following stencils are derived from the standard Finite Difference coefficients used in computational physics.

- Forward Difference (First Order, $O(h)$) $f'(x_i) \approx \frac{f_{i+1} - f_i}{h}$

- Backward Difference (First Order, $O(h)$) $f'(x_i) \approx \frac{f_i - f_{i-1}}{h}$

- Centered Difference (Second Order, $O(h^2)$) $f'(x_i) \approx \frac{f_{i+1} - f_{i-1}}{2h}$

- Three-Point Backward Difference (Second Order, $D_{-2}$) Used for indices $i$, $i-1$, $i-2$: $f'(x_i) \approx \frac{3f_i - 4f_{i-1} + f_{i-2}}{2h}$

- Centered Second Derivative (Second Order, $O(h^2)$) Derived from Table 2, FiniteDifferences.pdf: $f''(x_i) \approx \frac{f_{i+1} - 2f_i + f_{i-1}}{h^2}$